In [119]:
# import dependencies
from pytrends.request import TrendReq
import pandas as pd
import datetime as dt
from dateutil.relativedelta import relativedelta
import time

In [120]:
# initialize pytrends object
pytrend = TrendReq(hl='en-US', tz=360)

In [121]:
# pull in incomplete data
movie_df = pd.read_csv("output/incomplete_buzz_data.csv")
movie_df

,Unnamed: 0,Unnamed: 0.1,movie_id,title,movie_rating,release_date,runtime,genre_name,Director,Writer,Actors,Language,Poster,metascore,imdb_rating,boxoffice,buzz
0,0,0,0,Pirates of the Caribbean: Dead Man's Chest,PG-13,2006-07-07,151,"Action, Adventure, Fantasy",Gore Verbinski,"Ted Elliott, Terry Rossio, Stuart Beattie","Johnny Depp, Orlando Bloom, Keira Knightley","English, Turkish, Greek, Mandarin, French",https://m.media-amazon.com/images/M/MV5BMTcwOD...,53.0,7.3,423315812,3
1,1,1,1,Night at the Museum,PG,2006-12-22,108,"Adventure, Comedy, Family",Shawn Levy,"Robert Ben Garant, Thomas Lennon, Milan Trenc","Ben Stiller, Carla Gugino, Ricky Gervais","English, Italian, Hebrew",https://m.media-amazon.com/images/M/MV5BMTQyOT...,48.0,6.4,250863268,6
2,2,2,2,Cars,G,2006-06-09,117,"Animation, Adventure, Comedy","John Lasseter, Joe Ranft","John Lasseter, Joe Ranft, Jorgen Klubien","Owen Wilson, Bonnie Hunt, Paul Newman","English, Italian, Japanese, Yiddish",https://m.media-amazon.com/images/M/MV5BMTg5Nz...,73.0,7.1,244082982,4
3,3,3,3,X-Men: The Last Stand,PG-13,2006-05-26,104,"Action, Adventure, Sci-Fi",Brett Ratner,"Simon Kinberg, Zak Penn","Patrick Stewart, Hugh Jackman, Halle Berry",English,https://m.media-amazon.com/images/M/MV5BNDBhND...,58.0,6.7,234362462,23
4,4,4,4,The Da Vinci Code,PG-13,2006-05-19,149,"Mystery, Thriller",Ron Howard,"Akiva Goldsman, Dan Brown","Tom Hanks, Audrey Tautou, Jean Reno","English, French, Latin, Spanish",https://m.media-amazon.com/images/M/MV5BMjIxMj...,46.0,6.6,217536138,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4468,4469,4469,6880,Confetti,R,2006-05-05,100,"Comedy, Music, Romance",Debbie Isitt,Debbie Isitt,"Martin Freeman, Jessica Hynes, Stephen Mangan",English,https://m.media-amazon.com/images/M/MV5BMTM1MD...,57.0,5.7,32934,0
4469,4470,4470,6881,Bloody Hell,R,2021-01-14,93,"Action, Comedy, Horror",Alister Grierson,Robert Benjamin,"Ben O'Toole, Meg Fraser, Caroline Craig","English, Finnish",https://m.media-amazon.com/images/M/MV5BNThhZm...,52.0,6.6,32124,0
4470,4471,4471,6883,Yakuza Princess,R,2021-10-07,111,Thriller,Vicente Amorim,"Vicente Amorim, Danilo Beyruth, Kimi Lee","MASUMI, Jonathan Rhys Meyers, Tsuyoshi Ihara",English,https://m.media-amazon.com/images/M/MV5BMmFhM2...,34.0,4.4,30134,0
4471,4472,4472,6884,Monday,R,2021-04-16,116,"Drama, Romance",Argyris Papadimitropoulos,"Rob Hayes, Argyris Papadimitropoulos","Sebastian Stan, Denise Gough, Yorgos Pirpassop...","English, Greek",https://m.media-amazon.com/images/M/MV5BYTg4NW...,58.0,6.1,29262,0


In [122]:
# loop through df
for index, row in movie_df.iterrows():
    # starts where there is no buzz data
    if row["buzz"] > 0:
        continue
    # set up search parameters
    title = row["title"].split(': ')[0] + " movie"
    release_date = row["release_date"]
    start_date = (dt.datetime.strptime(release_date, '%Y-%m-%d') - relativedelta(years=1)).strftime("%Y-%m-%d")
    timeframe = f'{start_date} {release_date}'  
    # print(title, start_date, release_date)
    # builds query and sends request
    pytrend.build_payload(kw_list=[title], timeframe=timeframe)
    df = pytrend.interest_over_time()
    # makes sure the response isn't null
    # counts all weeks where buzz >= 20
    try:
        buzz = len(df.loc[df[title] >= 20])
    except KeyError:
        print("bad data", title)
        problem_entries.append(title)
        pass
    # saves score in buzz column
    movie_df.loc[index, "buzz"] = buzz
    print(title)
    # wait to avoid 429: too many requests error
    time.sleep(2)

ResponseError: The request failed: Google returned a response with code 429.

In [118]:
# save or overwrite file with new data
movie_df.to_csv("output/incomplete_buzz_data.csv")